Importando bibliotecas

In [1]:
import gsidiag as gd
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np

ModuleNotFoundError: No module named 'gsidiag'

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

from itertools import combinations
import seaborn as sns


Definindo parâmetros para o uso do readDiag

In [ ]:
DIRdiag = "/home/patrick/readDiag/data"

varName = "amsua"
varType = "n15"
dateIni="2020010100" 
dateFin="2020010106" 
nHour = "6"          
vminOMA = -2.0       
vmaxOMA = 2.0        
vminSTD = 0.0        
vmaxSTD = 14.0       
Level = 1000
Lay = None           
SingleL = "All" 

datei = datetime.strptime(str(dateIni), "%Y%m%d%H")
datef = datetime.strptime(str(dateFin), "%Y%m%d%H")
dates = [dates.strftime('%Y%m%d%H') for dates in pd.date_range(datei, datef,freq="6H").tolist()]

print(dates)


Encontrando e listando arquivos para serem usados

In [ ]:
paths, pathsc = [], []

OuterL = "01"        
[paths.append(DIRdiag+"/"+dt+"/diag_amsua_n15_"+OuterL+"."+dt) for dt in dates]

OuterLc = "03"
[pathsc.append(DIRdiag+"/"+dt+"/diag_amsua_n15_"+OuterLc+"."+dt) for dt in dates]

print(paths)
print("")
print(pathsc)

Lendo arquivos listado usando o readDiag, concatenando os dados e fazendo primeira separação de dados para treino e teste

In [ ]:
read = True

if read:        
    gdf_list = []
    print("")
    print("Aguarde, o tempo total estimado para a leitura dos arquivos é de "+
          str(int((float(len(paths))*20 )/60))+" minutos e "+
          str(int((float(len(paths))*20 )%60))+" segundos.")
    print("")
    for path, pathc in zip(paths,pathsc):
        print("Reading "+path)
        gdf = gd.read_diag(path,pathc)
        gdf_list.append(gdf)

    print(gdf_list)
    
    separator = " ====================================================================================================="

    df_concatenado = pd.DataFrame()
    
    for objeto in gdf_list:
        
        print(separator)
        print("Separando dados do arquivo" + str(objeto._diagFile))
        print(separator)
        
        
        dados_dict = {
                'lat': objeto.obsInfo[varName].loc[varType].lat,
                'lon': objeto.obsInfo[varName].loc[varType].lon,
                'elev': objeto.obsInfo[varName].loc[varType].elev,
                'nchan': objeto.obsInfo[varName].loc[varType].nchan,
                'time': objeto.obsInfo[varName].loc[varType].time,
                'iuse': objeto.obsInfo[varName].loc[varType].iuse,
                'idqc': objeto.obsInfo[varName].loc[varType].idqc,
                'inverr': objeto.obsInfo[varName].loc[varType].inverr,
                'oer': objeto.obsInfo[varName].loc[varType].oer,
                'obs': objeto.obsInfo[varName].loc[varType].obs,
                'omf': objeto.obsInfo[varName].loc[varType].omf,
                'omf_nobc': objeto.obsInfo[varName].loc[varType].omf_nobc,
                'emiss': objeto.obsInfo[varName].loc[varType].emiss,
                'oma': objeto.obsInfo[varName].loc[varType].oma,
                'oma_nobc': objeto.obsInfo[varName].loc[varType].oma_nobc,
                'imp': objeto.obsInfo[varName].loc[varType].imp,
                'dfs': objeto.obsInfo[varName].loc[varType].dfs
                }
        
        df_objeto = pd.DataFrame(dados_dict)
        
        df_concatenado = pd.concat([df_concatenado, df_objeto], ignore_index=True)
    
    print(separator)
    
    df_concatenado.dropna(inplace=True)
    
    print(df_concatenado)
    #canaisimp_df = canaisimp_df.loc[canais_df.index]
    
    #print(canais_df)
    #print(canaisimp_df)
    
    print(separator)
    
    atributos = ['nchan', 'omf', 'oma', 'dfs']
    
    X = df_concatenado[atributos]
    y = df_concatenado['imp']
    
    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print('Treino e Teste prontos')
    print(separator)
    
    

Treinamento completo do primeiro modelo de IA: Floresta Aleatória para Regressão (dados contínuos), e plotando resultados estatísticos do modelo.

Definição da função para discretizar os dados de impacto:

In [ ]:
# Função que retorna a categoria:
def categoria_de(valorRF):
    categoria = None
    if valorRF <= -0.25:
        categoria = "Impacto negativo alto"
    elif valorRF > -0.25 and valorRF < 0:
        categoria = "Impacto negativo baixo"
    elif valorRF == 0:
        categoria = "Impacto neutro"
    elif valorRF > 0 and valorRF < 0.25:
        categoria = "Impacto positivo baixo"
    elif valorRF >= 0.25:
        categoria = "Impacto positivo alto"
    return categoria

Segunda leitura dos arquivos, concatenando os dados e aplicando a função para discretizar os dados de impacto, ao final, os dados são separados para treino e teste.

In [ ]:
df_concatenado2 = pd.DataFrame()

for objeto in gdf_list:
        
    print(separator)
    print("Separando dados do arquivo" + str(objeto._diagFile))
    print(separator)
        
        
    dados_dict = {
            'lat': objeto.obsInfo[varName].loc[varType].lat,
            'lon': objeto.obsInfo[varName].loc[varType].lon,
            'elev': objeto.obsInfo[varName].loc[varType].elev,
            'nchan': objeto.obsInfo[varName].loc[varType].nchan,
            'time': objeto.obsInfo[varName].loc[varType].time,
            'iuse': objeto.obsInfo[varName].loc[varType].iuse,
            'idqc': objeto.obsInfo[varName].loc[varType].idqc,
            'inverr': objeto.obsInfo[varName].loc[varType].inverr,
            'oer': objeto.obsInfo[varName].loc[varType].oer,
            'obs': objeto.obsInfo[varName].loc[varType].obs,
            'omf': objeto.obsInfo[varName].loc[varType].omf,
            'omf_nobc': objeto.obsInfo[varName].loc[varType].omf_nobc,
            'emiss': objeto.obsInfo[varName].loc[varType].emiss,
            'oma': objeto.obsInfo[varName].loc[varType].oma,
            'oma_nobc': objeto.obsInfo[varName].loc[varType].oma_nobc,
            'imp': objeto.obsInfo[varName].loc[varType].imp,
            'dfs': objeto.obsInfo[varName].loc[varType].dfs
            }
        
    df_objeto = pd.DataFrame(dados_dict)
    
    df_objeto['imp_categ'] = df_objeto['imp'].apply(categoria_de)
        
    df_concatenado2 = pd.concat([df_concatenado2, df_objeto], ignore_index=True)
    

    
print(separator)
    
df_concatenado2.dropna(inplace=True)
    
print(df_concatenado2)

print(separator)
    
atributos = ['nchan', 'omf', 'oma', 'dfs']
    
X = df_concatenado2[atributos]
y = df_concatenado2['imp_categ']
    
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
print('Treino e Teste prontos')
print(separator)

Treinamento completo do segundo modelo de IA: Classificador usando Floresta Aleatória. Há prints da matriz de confusão e do relatório de classificação:

In [ ]:
# Treinar o modelo
modeloRFC = RandomForestClassifier()
modeloRFC.fit(X_train, y_train)
print('Modelo treinado!')
print(separator)
      
# Fazer previsões
y_pred = modeloRFC.predict(X_test)

# Avaliar o modelo
matriz_confusao = confusion_matrix(y_test, y_pred)
relatorio_classificacao = classification_report(y_test, y_pred)

print("Matriz de Confusão:")
print(matriz_confusao)

print("\nRelatório de Classificação:")
print(relatorio_classificacao)



Plotagem da matriz de confusão e importância dos recursos (atributos) para o modelo de classificação com árvore aleatória 

In [ ]:
# Define as classes
classes = ["Impacto positivo alto", "Impacto positivo baixo", "Impacto negativo baixo", "Impacto negativo alto"]

# Plotar a matriz de confusão

plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.title("Matriz de Confusão - RFC - Todos canais")
plt.xlabel("Valor Previsto")
plt.ylabel("Valor Real")
plt.savefig("MatrizConfusao_All_Ch.png", facecolor='white')
plt.show()


importances = modeloRFC.feature_importances_
std = np.std([tree.feature_importances_ for tree in modeloRFC.estimators_], axis=0)

forest_importances = pd.Series(importances, index=atributos)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Importâncias dos recursos usando MDI")
ax.set_ylabel("Diminuição média na impureza")
fig.tight_layout()
plt.savefig("FeatureImportance_All_Ch.png", facecolor='white')
plt.show()

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

y_score = modeloRFC.fit(X_train, y_train).predict_proba(X_test)


pair_list = list(combinations(np.unique(y), 2))
print(pair_list)

pair_scores = []
mean_tpr = dict()

for ix, (label_a, label_b) in enumerate(pair_list):
    a_mask = y_test == label_a
    b_mask = y_test == label_b
    ab_mask = np.logical_or(a_mask, b_mask)

    a_true = a_mask[ab_mask]
    b_true = b_mask[ab_mask]

    idx_a = np.flatnonzero(label_binarizer.classes_ == label_a)[0]

    if label_b in label_binarizer.classes_:
        idx_b = np.flatnonzero(label_binarizer.classes_ == label_b)[0]
        # Continua o restante do seu código aqui
    else:
        print(f"A classe {label_b} não está presente nos dados de teste.")

    fpr_a, tpr_a, _ = roc_curve(a_true, y_score[ab_mask, idx_a])
    fpr_b, tpr_b, _ = roc_curve(b_true, y_score[ab_mask, idx_b])

    # Calcular a faixa de valores de FPR observados nas curvas ROC
    fpr_min = min(np.min(fpr_a), np.min(fpr_b))
    fpr_max = max(np.max(fpr_a), np.max(fpr_b))
    fpr_grid = np.linspace(fpr_min, fpr_max, 100)  # Ajuste o número de pontos conforme necessário
    
    mean_tpr[ix] = np.zeros_like(fpr_grid)
    mean_tpr[ix] += np.interp(fpr_grid, fpr_a, tpr_a)
    mean_tpr[ix] += np.interp(fpr_grid, fpr_b, tpr_b)
    mean_tpr[ix] /= 2
    mean_score = auc(fpr_grid, mean_tpr[ix])
    pair_scores.append(mean_score)

    # Obtenha os nomes das classes do LabelBinarizer
    target_names = label_binarizer.classes_
    
    fig, ax = plt.subplots(figsize=(6, 6))
    plt.plot(
        fpr_grid,
        mean_tpr[ix],
        label=f"Média {label_a} vs {label_b} (AUC = {mean_score :.2f})",
        linestyle=":",
        linewidth=4,
    )
    RocCurveDisplay.from_predictions(
        a_true,
        y_score[ab_mask, idx_a],
        ax=ax,
        name=f"{label_a} como classe positiva",
    )
    RocCurveDisplay.from_predictions(
        b_true,
        y_score[ab_mask, idx_b],
        ax=ax,
        name=f"{label_b} como classe positiva",
    )
    
    ax.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Chance Level')
    
    ax.set(
        xlabel="Taxa de falso positivo",
        ylabel="Taxa de verdade positiva",
        title=f"{target_names[idx_a]} vs {label_b} curva ROC",
    )
    
    plt.savefig(f'{target_names[idx_a]} vs {label_b}_curvaROC.png', facecolor='white')

print(f"Macro-averaged One-vs-One ROC AUC score:\n{np.average(pair_scores):.2f}")

In [2]:
# Define as classes
classes = ["Impacto positivo alto", "Impacto positivo baixo", "Impacto negativo baixo", "Impacto negativo alto"]

# Supondo que 'df' seja o seu DataFrame e 'canal' seja a coluna que contém os canais de radiância
canais = df_concatenado2['nchan'].unique()  # Obtém a lista de canais únicos

print(canais)

# Para cada canal, divida o DataFrame e treine um modelo

atributos = ['omf', 'oma', 'dfs', 'oer', 'obs', 'oma_nobc', 'omf_nobc', 'lat', 'lon', 'emiss']

modeloscanais = []
resultados = []


for canal in canais:
    # Divida o DataFrame para o canal atual
    df_canal = df_concatenado2[df_concatenado2['nchan'] == canal]
    
    # Verifique se a coluna 'canal' está presente
    if 'nchan' in df_canal.columns:
        # Separe os recursos (X) e o alvo (y)
        
        
        X = df_canal[atributos]
        y = df_canal['imp_categ']
        
        # Divida os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Treine o modelo para o canal atual
        nome_modelo = f"modeloch{canal}"
        modeloch = RandomForestClassifier()
        modeloscanais.append((nome_modelo, modeloch))
        modeloch.fit(X_train, y_train)
        
        # Faça previsões
        y_pred = modeloch.predict(X_test)
        
        # Avalie o modelo e imprima a matriz de confusão
        matriz_confusao = confusion_matrix(y_test, y_pred)
        print(f"Matriz de Confusão para o canal {canal}:")
        #print(matriz_confusao)
    else:
        print(f"Coluna 'canal' não encontrada para o canal {canal}. Verifique se os dados estão formatados corretamente.")
        
    precisao_media = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['precision']
    resultados.append((canal, precisao_media))
    
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(8, 6))
    sns.heatmap(matriz_confusao, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title(f"Matriz de Confusão (RFC) do canal {canal}")
    plt.xlabel("Valor Previsto")
    plt.ylabel("Valor Real")
    plt.savefig(f"MatrizConfusao_Channel_{canal}.png", facecolor='white')
    plt.show()

    
    importances = modeloch.feature_importances_
    std = np.std([tree.feature_importances_ for tree in modeloch.estimators_], axis=0)

    forest_importances = pd.Series(importances, index=atributos)

    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title(f"Importâncias dos recursos usando MDI - Canal {canal}")
    ax.set_ylabel("Diminuição média na impureza")
    fig.tight_layout()
    
    plt.savefig(f"FeatureImportance_Channel_{canal}.png", facecolor='white')
    plt.show()
    
    relatorio_classificacao = classification_report(y_test, y_pred)

    print(f"\nRelatório de Classificação para o canal {canal}:")
    print(relatorio_classificacao)
    
resultados_ordenados = sorted(resultados, key=lambda x: x[1], reverse=True)
for canal, precisao_media in resultados_ordenados:
    print(f"Modelo do canal: {canal}")
    print(f"Precisão Média: {precisao_media}")
    print("\n")

NameError: name 'df_concatenado2' is not defined

In [ ]:
'''from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

y_score = modelo.fit(X_train, y_train).predict_proba(X_test)

import matplotlib.pyplot as plt

from sklearn.metrics import RocCurveDisplay

display = RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
    plot_chance_level=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)",
)'''

In [ ]:
'''from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve, auc

# Ajuste LabelBinarizer aos rótulos de treinamento
label_binarizer = LabelBinarizer().fit(y_train)

# Transforme os rótulos de teste em um formato binário
y_onehot_test = label_binarizer.transform(y_test)

# Treine o modelo e obtenha as probabilidades das previsões
y_score = modelo.fit(X_train, y_train).predict_proba(X_test)

# Obter as classes do LabelBinarizer
classe = label_binarizer.classes_

# Exibir as classes
print("Classes presentes na lista:")
for c in classe:
    print(c)

# Escolha a classe de interesse e obtenha o índice correspondente
class_of_interest = 'Impacto positivo baixo'  # substitua pela classe de interesse
class_id = label_binarizer.classes_.tolist().index(class_of_interest)

# Calcule a curva ROC para a classe de interesse
fpr, tpr, _ = roc_curve(y_onehot_test[:, class_id], y_score[:, class_id])
roc_auc = auc(fpr, tpr)

# Crie a exibição da curva ROC
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=f"{class_of_interest} vs the rest")

# Plote a curva ROC
display.plot()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('One-vs-Rest ROC curves')
plt.show()'''

In [ ]:
atributos = ['nchan', 'omf', 'oma', 'dfs']
  
print(df_concatenado2[atributos])
print(df_concatenado2['imp_categ'])

X = df_concatenado2[atributos]
y = df_concatenado2['imp_categ']

# Codificar os rótulos se necessário
#label_encoder = LabelEncoder()
#y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train, X_test = X[:2000], X[2000:]
#y_train, y_test = y[:2000], y[2000:]

print('Gradiente Boosting Classifier')
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=5, random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

score = clf.score(X_test, y_test)
print("Acurácia:", score)

matriz_confusao = confusion_matrix(y_test, y_pred)
# Define as classes
classes = ["Impacto positivo alto", "Impacto positivo baixo", "Impacto negativo baixo", "Impacto negativo alto"]

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.title("Matriz de Confusão - GBC - Todos canais")
plt.xlabel("Valor Previsto")
plt.ylabel("Valor Real")
plt.savefig("MatrizConfusao_All_Ch.png", facecolor='white')
plt.show()

clf.feature_importances_

relatorio_classificacao = classification_report(y_test, y_pred)

print(f"\nRelatório de Classificação:")
print(relatorio_classificacao)


In [ ]:
atributos = ['nchan', 'omf', 'oma', 'dfs']
    
X = df_concatenado2[atributos]
y = df_concatenado2['imp_categ']



clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
    voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
from sklearn.model_selection import learning_curve
atributos = ['nchan', 'omf', 'oma', 'dfs']
    
X = df_concatenado2[atributos]
y = df_concatenado2['imp_categ']

modeloMLPC = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
modeloMLPC.fit(X, y)

y_pred = modeloMLPC.predict(X_test)

score = modeloMLPC.score(X_test, y_test)
print("Acurácia:", score)

matriz_confusao = confusion_matrix(y_test, y_pred)
# Define as classes
classes = ["Impacto positivo alto", "Impacto positivo baixo", "Impacto negativo baixo", "Impacto negativo alto"]

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.title("Matriz de Confusão - MLPC - Todos canais")
plt.xlabel("Valor Previsto")
plt.ylabel("Valor Real")
plt.savefig("MatrizConfusao_All_Ch.png", facecolor='white')
plt.show()

relatorio_classificacao = classification_report(y_test, y_pred)

print(f"\nRelatório de Classificação:")
print(relatorio_classificacao)


In [ ]:
'''# Calcule os tamanhos de treinamento e as pontuações de treinamento/teste
train_sizes, train_scores, test_scores = learning_curve(
    modeloMLPC, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 100))

# Calcule as médias e desvios padrão das pontuações de treinamento/teste
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure()
plt.title("Curva de Aprendizado (MLPClassifier)")
plt.xlabel("Tamanho do Conjunto de Treinamento")
plt.ylabel("Pontuação")
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Pontuação de Treinamento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Pontuação de Teste")
plt.legend(loc="best")
plt.show()'''